In [ ]:
print("Hello")

### Read whatsapp data

In [ ]:
import re
import pandas as pd

In [ ]:
f = open('WhatsApp Chat with গণরুম.txt','r',encoding='utf-8')

In [ ]:
data=f.read()

In [ ]:
print(data)

In [ ]:
print(type(data))

In [ ]:
data = data.replace('\u202f', ' ')

In [ ]:
data

# Data Pre-Processing

### convert this data into list

In [ ]:
pattern = r'(\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}(?:\s?[ap]m)?)\s-\s'

In [ ]:
messages = re.split(pattern, data)[1:]
len(messages)

In [ ]:
messages

### split messages & extract dates

In [ ]:
dates = messages[0::2]
texts = messages[1::2]

In [ ]:
dates

In [ ]:
texts

### dataframe

In [ ]:
df = pd.DataFrame({
    'message_date': dates,
    'user_message': texts
})

### datetime conversion

In [ ]:
df['message_date'] = pd.to_datetime(
    df['message_date'],
    format='%d/%m/%y, %I:%M %p'
)

rename column

In [ ]:
df.rename(columns={'message_date': 'date'}, inplace=True)

In [ ]:
df.tail()

In [ ]:
df.shape

### Extract user from messages into new column

In [ ]:
users = []
msgs = []

for msg in df['user_message']:
    entry = re.split(r':\s', msg, maxsplit=1)
    if len(entry) == 2:
        users.append(entry[0])
        msgs.append(entry[1])
    else:
        users.append('group_notification')
        msgs.append(entry[0])

df['user'] = users
df['message'] = msgs

df.drop(columns=['user_message'], inplace=True)


In [ ]:
df.sample(10)

### Extract year,month,day,hour and minute from 'date' column

In [ ]:
df['date'].dt.year.head()

In [ ]:
df['year'] = df['date'].dt.year

In [ ]:
df.head()

In [ ]:
df['date'].dt.month_name().sample(10)

In [ ]:
df['month'] = df['date'].dt.month_name()

In [ ]:
df.sample(10)

In [ ]:
df['date'].dt.day.sample(10)

In [ ]:
df['day'] = df['date'].dt.day

In [ ]:
df['hour'] = df['date'].dt.hour

In [ ]:
df['minute'] = df['date'].dt.minute

In [ ]:
df.sample(10)

In [ ]:
df[df['user'] == 'Tamim Hossain'].shape

### Extract Messages and words in app.py and here i make demo

In [ ]:
words = []
for message in df['message']:
    #print(message.split())
    words.extend(message.split())

In [ ]:
len(words)

In [ ]:
df.shape[1]

### Extract media (test)

In [ ]:
df[df['message'] == '<Media omitted>']

In [ ]:
df[df['message'] == '<Media omitted>/n']

In [ ]:
df['message'].iloc[148]

In [ ]:
df[df['message'] == '<Media omitted>\n'].sample(20)

In [ ]:
df[df['message'] == '<Media omitted>\n'].shape

### Extract link test

In [ ]:
from urlextract import URLExtract # if pass any url messages in this package, it return us only this link

In [ ]:
extractor = URLExtract()
urls = extractor.find_urls("Lets have URL have stackoverflow.com as an example google.com, http://facebook.com, ftp://url.in")
urls

In [ ]:
links = []
for message in df.message:
    links.extend(extractor.find_urls(message))

In [ ]:
links

In [ ]:
len(links)

### Extract top 5 busy users

In [ ]:
df['user'].value_counts()

In [ ]:
x = df['user'].value_counts().head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x

In [ ]:
name = x.index
count = x.values

In [ ]:
plt.bar(name,count)
plt.xticks(rotation='vertical')

### An 'user' how many send messages

In [ ]:
df['user'].value_counts()/df.shape[0]

In [ ]:
(df['user'].value_counts()/df.shape[0])*100

In [ ]:
round((df['user'].value_counts()/df.shape[0])*100,2).reset_index()

In [ ]:
round((df['user'].value_counts()/df.shape[0])*100,2).reset_index().rename(columns={'user': 'name', 'count': 'percent'})

### I want that a group which 20/25 words most used

In [ ]:
words = []

for message in df['message']:
    #print(message)
    words.extend(message.split())

In [ ]:
words

In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

### Now
- remove 'group notification'
- remove 'media omitted message'
- remove stop words

In [ ]:
df[df['user'] == 'group_notification']

In [ ]:
df[df['user'] != 'group_notification']

In [ ]:
temp = df[df['user'] != 'group_notification']

In [ ]:
temp

In [ ]:
temp = temp[temp['message'] != '<Media omitted>\n']

In [ ]:
temp

In [ ]:
words = []

for message in temp['message']:
    words.extend(message.split())

In [ ]:
f = open('stopwords.txt','r')
banglish_stopwords = f.read()

In [ ]:
words = []

for message in temp['message']:
    for word in message.lower().split():
        if word not in banglish_stopwords:
            words.append(word)

In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

## Emoji analysis

In [ ]:
import emoji

In [ ]:
emojis = []

for message in df['message']:
    emojis.extend([c for c in message if emoji.is_emoji(c)])

In [ ]:
emojis

In [ ]:
most_common_emoji = Counter(emojis).most_common(10)

In [ ]:
most_common_emoji

In [ ]:
most_common_emoji_dataframe = pd.DataFrame(Counter(emojis).most_common(10))

In [ ]:
most_common_emoji_dataframe

## Analyzing Message Timeline

In [ ]:
df

In [ ]:
df['month_num'] = df['date'].dt.month

In [ ]:
df.sample(5)

In [ ]:
df.groupby(['year','month_num','month']).count()['message'].reset_index()

In [ ]:
timeline = df.groupby(['year','month_num','month']).count()['message'].reset_index()

In [ ]:
timeline

In [ ]:
time = []
for i in range(timeline.shape[0]):
    time.append(timeline['month'][i] + "-" + str(timeline['year'][i]))

In [ ]:
time

In [ ]:
timeline['time'] = time

In [ ]:
timeline

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(timeline['time'],timeline['message'])
plt.xticks(rotation=90)
plt.show()

#### now check by date

In [ ]:
df['only_date'] = df['date'].dt.date

In [ ]:
df

In [ ]:
df.groupby('only_date').count()['message'].reset_index()

In [ ]:
daily_timeline = df.groupby('only_date').count()['message'].reset_index()

In [ ]:
daily_timeline

In [ ]:
daily_timeline.tail()

In [ ]:
plt.figure(figsize=(18,10))
plt.plot(daily_timeline['only_date'], daily_timeline['message'])